# Michaelis-Menten Model Calibration Notebook

Based on PTemPest example written in Matlab [here](https://github.com/RuleWorld/ptempest/tree/master/examples/michment)

## Overview

This system describes the 1) reversible binding of an enzyme to substrate and 2) production of substrate product which is defined in the following scheme:  
$$E + S \rightleftharpoons^{k_f}_{k_r} ES \longrightarrow^{k_{cat}} E + P$$  

Assuming total enzyme concentration is significantly smaller than substrate concentration (i.e., $[E]_T \ll [S]$), the rate is defined as:  
$$\frac{d[P]}{dt} = \frac{k_{cat}[E]_T[S]}{K_M + [S]}$$
where $K_M = \frac{k_{cat} + k_r}{k_f}$

## Model Calibration

The following notebook calibrates the Michaelis-Menten model system using synthetically generated data with 1% Gaussian error. Here, we test the following inference methods:
1. Metropolis-Hastings (`pyPESTO`)
2. Parallel-Tempering MCMC (`pyPESTO`)
3. Nested Sampling (`dynesty`)
4. Sequential Monte Carlo (`pocoMC`)
5. Preconditioned Monte Carlo (`pocoMC`)

### Load relevant packages

In [1]:
import os
import time
import roadrunner
import numpy as np
import pandas as pd

from scipy.stats import qmc
from multiprocessing import Pool

import pocomc as pc
import dynesty as dy
import pypesto as pype
import pypesto.engine as eng
import pypesto.sample as sample
import pypesto.store as store
import pypesto.optimize as optimize
from pypesto.ensemble import Ensemble

### Defining the `Model` class

The model class has the following attributes:
1. `x_n` : `int`
    Number of species in the model
2. `fit_x0` : `bool`
    Whether the initial conditions are to be estimated and are therefore include in `theta` args.
3. `x0` : `list[float], optional`
    The initial conditions of model species
4. `theta_n` : `int`
    Number of parameters to be fit. This includes ALL parameters to be estimated which MAY include initial conditions and the standard deviation of the model species. The order of the model parameters in this list is assumed to be as follows:
    1. ODE equation parameters
    2. Initial conditions (denoted $x_\#$, **optional**)
    3. Species standard deviations (denoted $\sigma_\#$)
5. `theta_true` : `list[float]` of shape `(theta_n)`
    True theta values of the model system. The order of 
6. `theta_names` : `list[str]` of shape `(theta_n)`
    Name of parameters for plotting purposes
7. `lower_bnds` : `list[float]` of shape `(theta_n)`
    Lower bounds of parameter values 
8. `upper_bnds` : `list[float]` of shape `(theta_n)`
    Upper bounds of parameter values 
9. `ts` : `list[float]` 
    Experimental data times 
10. `data` : `list[float]` of shape `(x_n, ts)` 
    Experimental data used for model calibration
11. `sys_fun` : `callable` 
    Model function used to solve system ODEs. It will be called by `solve_ivp`.

In [2]:
class Model:
    def __init__(self, opts): #initial settings
        for key in opts: #loops for all labels in the list 'key'
            setattr(self, key, opts[key]) #creates a dictionary where 'key' are the list of labels & 'ops[key]' are the values

    def __call__(self, theta_new):
        theta_new = theta_new
        res = self.log_likelihood(theta_new)
        return res
    
    def run_sim(self, model_param = None, x0=None): #takes in canidate parameters then solves the ode
        if model_param is None:
            model_param= self.theta_true[:self.ODE_params_n]  #sets the model_params to just the model parameters
        if x0 is None:
            x0 = self.x0 #if x0 not defined, default x0 to the model x0
        t_span = (self.ts[0], self.ts[-1]) #define the time span
        x_n = self.x_n 
        
        result = solve_ivp(self.sys_fun, t_span, y0=x0, 
                        t_eval=self.ts, args=([model_param])) #solve ODE (with model parameters)
        
        return result #returns result.t and result.y
    
    def log_prior(self, theta_new): 
        bools = [(low <= i <= high) for i,low,high in zip(theta_new, self.lower_bnds, self.upper_bnds)] #if generated values are within bounds
        all_in_range = np.all(bools) #if all values are true, then output is true
        if all_in_range: #if true
            return 0.0 #give 0
        return -np.inf #if even one parameter out of bounds, it's false, and returns -infinity

    def log_likelihood(self, theta_new): #how good is this canidate parameter fitting my data (maximize it)
        model_param = theta_new[:self.ODE_params_n] 
        if self.fit_x0: 
            x0 = theta_new[self.ODE_params_n:(self.ODE_params_n + self.x_n)] #sets x0 to 'theta_true' x0 values
        else:
            x0 = self.x0

        if self.fit_sigma:
            sigma = theta_new[-len(self.observable_index):] #observable index related to sigma
        else:
            sigma = [1] * len(self.observable_index) #makes all sigmas default to 1

        y = self.run_sim(model_param=model_param, x0= x0).y #sets y to the y results of solving ODE
        data = self.data #sets data
        if self.x_n > 1:
            y = np.transpose(y)

        # Calculate posterior; how good is parameter in terms of fitting the data
        term1 = -0.5 * np.log(2*np.pi*np.square(sigma))
        term2 = np.square(np.subtract(y, data)) / (2*np.square(sigma))
        logLH = np.sum(term1 - term2)
        return logLH